# Create deep links

Below is a widget for creating deep links to start collecting patient data with CommonHealth Cloud

Choices to make:


- Select the Partner application the data will be shared with.

  We currently have two partner applications for collecting data: 'testing' for testing, and the Pre-MVP itself.
  Use 'testing' if you are still working things out, like testing this widget page, or 'Pre-MVP' if you want to create a deep link that will add data to production Pre-MVP demos.
- Select the patient id. This is the id you will use to fetch the patient's uploaded data in the future.

Additional choices where you likely don't want to change the defaults:
- Select a subset of data types to request (default: heart rate and blood pressure)
- Select the expiration of the deep link (there is little reason to change this from the default).
  The patient must follow the link before this many days, or you will need to send them a new link.



In [ ]:
import html
from textwrap import dedent

import ipywidgets as W
from IPython.display import HTML, display
from jupyter_health import JupyterHealthCHClient

ch_clients = {key: JupyterHealthCHClient(key) for key in ("testing", "prod")}

available_scopes = (
    "OMHealthResource.HeartRate.Read",
    "OMHealthResource.BloodPressure.Read",
)
_width = "400px"


@W.interact(
    partner=W.Dropdown(
        options={
            ch_client.get_configuration()["name"]: key
            for key, ch_client in ch_clients.items()
        },
        value="testing",
        layout=W.Layout(width=_width),
    ),
    patient_id="",
    expires_days=W.IntSlider(value=30, min=1, max=30),
    scopes=W.SelectMultiple(
        options=available_scopes,
        value=available_scopes,
        layout=W.Layout(width="400px"),
        description="scopes",
    ),
).options(manual=True, manual_name="Generate deep link")
def create_deep_link(partner, patient_id, expires_days, scopes):
    """Create a deep link with inputs"""
    if not patient_id:
        print("patient id is required")
        return

    if not scopes:
        print("Must specify at least one scope to request")
        return

    ch_client = ch_clients[partner]
    deeplink = ch_client.construct_authorization_request_deeplink(
        patient_id,
        scope=" ".join(scopes),
        expiration_seconds=expires_days * (24 * 3600),
    )
    display(
        HTML(
            dedent(
                f"""
    <h2>Deep link for patient: {html.escape(patient_id)}</h2>

    <p>
    Share this link with the patient to start collecting data:
    </p>
    
    <pre id="jupyterhealth-deeplink">{html.escape(deeplink)}</pre>
    """
                # add button to copy link
                + """
                        
    <button id="copy-button" onclick="copyLink()" class="jupyter-button">Copy link</button>

    <script type="text/javascript">
    function copyLink() {
      // Copy the deep link
      var copyText = document.getElementById("jupyterhealth-deeplink");
      var copyRange = document.createRange();
      copyRange.selectNode(copyText);
      window.getSelection().removeAllRanges();
      window.getSelection().addRange(copyRange);
      document.execCommand("copy");
      document.getElementById("copy-button").innerText = "copied!";
    }
    </script>
    """
            )
        )
    )